In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import time
from functions import basic
from functions import interaction
from functions import evolution
import main
%matplotlib inline

In [2]:
%%javascript
require(["codemirror/keymap/sublime", "notebook/js/cell", "base/js/namespace"],
    function(sublime_keymap, cell, IPython){
        cell.Cell.options_default.cm_config.keyMap = 'sublime';
        var cells = IPython.notebook.get_cells();
        for(var cl=0; cl< cells.length ; cl++){
            cells[cl].code_mirror.setOption('keyMap', 'sublime');
        }
    }
);

<IPython.core.display.Javascript object>

In [3]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

In [ ]:
# reading input file
filename = "fcc100a256.txt"
lattice = basic.read_file(filename)

In [ ]:
# computing ditances, finding neighbours

# no PBC test
start = time.process_time()
for i in range(100):
    mask,distances = basic.find_neighbours(*lattice)
print(f'no PBC - 100 iterations; computing time: {time.process_time() - start :.2f} seconds')

start = time.process_time()
for i in range(100):
    mask,distances = basic.find_neighbours(*lattice, PBC=True)
print(f'PBC - 100 iterations; computing time: {time.process_time() - start :.2f} seconds')

In [ ]:
# calculating potential energy
mask,distances = basic.find_neighbours(*lattice)

# approximate LJ
start = time.process_time()
for i in range(100):
    LJA = interaction.lennard_jones_approx(distances)
print(f'approximate LJ - 100 iterations; computing time: {time.process_time() - start :.2f} seconds')

# true LJ
start = time.process_time()
for i in range(100):
    LJ = interaction.lennard_jones(distances)
print(f'true LJ - 100 iterations; computing time: {time.process_time() - start :.2f} seconds')

print('')
print(f'approximate LJ: {LJA:.2f}')
print(f'true LJ: {LJ:.2f}')

In [ ]:
# calculating forces

# approximate LJ
start = time.process_time()
for i in range(100):
    FxA,FyA,FzA = interaction.calc_force_approx(*lattice,distances,PBC=True)
print(f'approximate LJ - 100 iterations; computing time: {time.process_time() - start :.2f} seconds')

# approximate LJ
start = time.process_time()
for i in range(100):
    Fx,Fy,Fz = interaction.calc_force(*lattice,distances,PBC=True)
print(f'true LJ - 100 iterations; computing time: {time.process_time() - start :.2f} seconds')

print('')
print(f'approximate LJ, force on 0-th atom: {FxA[1]:.2f},{FyA[1]:.2f},{FzA[1]:.2f}')
print(f'true LJ, force on 0-th atom: {Fx[1]:.2f},{Fy[1]:.2f},{Fz[1]:.2f}')

In [ ]:
n_atoms,sx,sy,sz,x,y,z = lattice

fig,axs = plt.subplots(1,3,sharey=True,figsize=(12,4))

for i in range(255):
    axs[0].plot(x[i],FxA[i],'.')
    axs[1].plot(y[i],FyA[i],'.')
    axs[2].plot(z[i],FzA[i],'.')

for i in range(3):
    axs[i].grid()

axs[0].set_xlabel('x')
axs[0].set_ylabel('Fx')
axs[1].set_xlabel('y')
axs[1].set_ylabel('Fy')
axs[2].set_xlabel('z')
axs[2].set_ylabel('Fz')

fig.suptitle('Force components - perfect lattice - approximate LJ - PBC')
plt.show()

In [ ]:
fig,axs = plt.subplots(1,3,sharey=True,figsize=(12,4))

for i in range(255):
    axs[0].plot(x[i],Fx[i],'.')
    axs[1].plot(y[i],Fy[i],'.')
    axs[2].plot(z[i],Fz[i],'.')

for i in range(3):
    axs[i].grid()

axs[0].set_xlabel('x')
axs[0].set_ylabel('Fx')
axs[1].set_xlabel('y')
axs[1].set_ylabel('Fy')
axs[2].set_xlabel('z')
axs[2].set_ylabel('Fz')

fig.suptitle('Force components - perfect lattice - true LJ - PBC')
plt.show()

In [ ]:
# initializing speed
m_ag = 108*1.66e-27/16
kb = 1/11603
T = 100

start = time.process_time()
vx,vy,vz = basic.initialize_speed(n_atoms,x,y,z,T,remove_translation=True)
print(f'speed initialization; computing time: {time.process_time() - start :.3f} seconds\n')

v2 = vx**2 + vy**2  + vz**2
Ekin = 0.5*m_ag*np.sum(v2)
Tkin = 2*Ekin/(3*n_atoms*kb)  

print('translation removal check:')
print(f'    mean vx: {np.mean(vx)}')
print(f'    mean vy: {np.mean(vy)}')
print(f'    mean vz: {np.mean(vz)}\n')
print(f'Set temperature: {T:.2f}K; real temperature: {Tkin:.2f}K')

### Full simulation using built-in function

#### No PBC, no approx

In [ ]:
filename = "fcc100a256.txt"
timelength = 50e-12    
timestep = 1.5e-14   
T = 50

results = main.make_simulation(filename,T,timelength,timestep)
time_array,all_x,all_y,all_z,Temp_array,energy_array = results

In [ ]:
print(f'DeltaE/E: {np.std(energy_array[2000:])/np.mean(energy_array[2000:])}')

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(12,4))

axs[0].plot(time_array*1e12,Temp_array,'.-')
axs[0].plot(time_array[2000:]*1e12,Temp_array[2000:],'.-')
axs[1].plot(time_array*1e12,energy_array,'.-')
axs[1].plot(time_array[2000:]*1e12,energy_array[2000:],'.-')

for ax in axs:
    ax.set_xlabel('Time [ps]')

axs[0].set_ylabel('Temperature [K]')
axs[1].set_ylabel('Energy [eV]')

fig.tight_layout()

In [ ]:
fig,axs = plt.subplots(1,3,sharey=True,figsize=(12,4))

axs[0].plot(time_array*1e12,all_x[0],'.-')
axs[1].plot(time_array*1e12,all_y[0],'.-')
axs[2].plot(time_array*1e12,all_z[0],'.-')

for ax in axs:
    ax.set_xlabel('Time [ps]')
    
axs[0].set_ylabel('x[0] [nm]')
axs[1].set_ylabel('y[0] [nm]')
axs[2].set_ylabel('z[0] [nm]')

fig.tight_layout()

#### PBC, approx

In [ ]:
filename = "fcc100a256.txt"
timelength = 40e-12    
timestep = 1.5e-14   
T = 100

results = main.make_simulation(filename,T,timelength,timestep,PBC=True,approx=True)
time_array,all_x,all_y,all_z,Temp_array,energy_array = results

In [ ]:
print(f'DeltaE/E: {np.std(energy_array[400:])/np.mean(energy_array[400:])}')

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(12,4))

axs[0].plot(time_array*1e12,Temp_array,'.-')
axs[0].plot(time_array[400:]*1e12,Temp_array[400:],'.-')
axs[1].plot(time_array*1e12,energy_array,'.-')
axs[1].plot(time_array[400:]*1e12,energy_array[400:],'.-')

for ax in axs:
    ax.set_xlabel('Time [ps]')

axs[0].set_ylabel('Temperature [K]')
axs[1].set_ylabel('Energy [eV]')

fig.tight_layout()

In [ ]:
fig,axs = plt.subplots(1,3,sharey=True,figsize=(12,4))

axs[0].plot(time_array*1e12,all_x[0],'.-')
axs[1].plot(time_array*1e12,all_y[0],'.-')
axs[2].plot(time_array*1e12,all_z[0],'.-')

for ax in axs:
    ax.set_xlabel('Time [ps]')
    
axs[0].set_ylabel('x[0] [nm]')
axs[1].set_ylabel('y[0] [nm]')
axs[2].set_ylabel('z[0] [nm]')

fig.tight_layout()

### Full simulation with built-in functions

In [ ]:
filename = "fcc100a256.txt"
timelength = 12e-12    
timestep = 1.5e-14   
T = 100

lattice = basic.read_file(filename)
results = main.make_simulation(filename,T,timelength,timestep,PBC=True,approx=True)
time_array,all_x,all_y,all_z,Temp_array,energy_array = results

In [ ]:
dump_time = 5e-12
df50 = main.build_results_df(T/2,lattice,results,dump_time)

In [ ]:
df50.head()

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(12,4))

for ix,(z0,z0_group) in enumerate(df50.groupby('z0')):
    axs[0].plot(z0_group['z0']-df50['z0'].mean(),z0_group['z_mean']-z0_group['z0'],'.')
    axs[1].plot(z0_group['z0']-df50['z0'].mean(),z0_group['z_std'],'.')
    
for ax in axs:
    ax.set_xlabel('z-distance from mass center [A°]')
axs[0].set_ylabel('displacement from starting position [A°]')
axs[0].set_ylabel('oscillation amplitude [A°]')
fig.tight_layout()

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(12,4))

for ix,(z0,z0_group) in enumerate(df50.groupby('z0')):
    axs[0].errorbar(z0_group['z0'].mean()-df50['z0'].mean(),z0_group['z_mean'].mean()-z0_group['z0'].mean(),
                    yerr=z0_group['z_mean'].std(),fmt='.-',capsize=3)
    axs[1].errorbar(z0_group['z0'].mean()-df50['z0'].mean(),z0_group['z_std'].mean(),
                    yerr=z0_group['z_std'].std(),fmt='o-',capsize=3)
    
for ax in axs:
    ax.set_xlabel('z-distance from mass center [A°]')
axs[0].set_ylabel('displacement from starting position [A°]')
axs[1].set_ylabel('oscillation amplitude [A°]')
fig.tight_layout()

### More simulations in a row, saving results

In [8]:
import json
configs = pd.read_json('configs.json')

In [9]:
print(configs.to_string())

                                                                                         simulation_list
0                 {'temperature': 100, 'timestep': 1.5e-14, 'total_time': 2e-11, 'rejected_time': 8e-12}
1                   {'temperature': 200, 'timestep': 1e-14, 'total_time': 2e-11, 'rejected_time': 8e-12}
2                   {'temperature': 400, 'timestep': 5e-15, 'total_time': 2e-11, 'rejected_time': 8e-12}
3  {'temperature': 800, 'timestep': 2.5000000000000004e-15, 'total_time': 2e-11, 'rejected_time': 8e-12}


In [ ]:
filename = "fcc100a256.txt"

for config in configs.simulation_list:
    temperature = config['temperature']
    timestep = config['timestep']
    total_time = config['total_time']
    rejected_time = config['rejected_time']
    lattice = basic.read_file(filename)
    results = main.make_simulation(filename,temperature,total_time,timestep,PBC=True,approx=True)
    df = main.build_results_df(temperature,lattice,results,rejected_time)
    df.to_excel(f'results/results_T{temperature}.xlsx')

Translation removal check:
    mean vx: 3.814697265625e-05
    mean vy: -4.57763671875e-05
    mean vz: 0.00022125244140625

Set temperature: 100.00K; real temperature: 100.00K 



  0%|          | 0/1333 [00:00<?, ?it/s]

Translation removal check:
    mean vx: 6.866455078125e-05
    mean vy: 7.62939453125e-06
    mean vz: -0.0003662109375

Set temperature: 200.00K; real temperature: 200.00K 



  0%|          | 0/1999 [00:00<?, ?it/s]

Translation removal check:
    mean vx: -0.0001220703125
    mean vy: 0.00020599365234375
    mean vz: -0.0004730224609375

Set temperature: 400.00K; real temperature: 400.00K 



  0%|          | 0/3999 [00:00<?, ?it/s]

Translation removal check:
    mean vx: -3.0517578125e-05
    mean vy: -0.0001354217529296875
    mean vz: -5.340576171875e-05

Set temperature: 800.00K; real temperature: 800.00K 



  0%|          | 0/7999 [00:00<?, ?it/s]

### Test zone - used for quick tests and monkey patch